# Data Compressed
An adversary may compress data (e.g., sensitive documents) that is collected prior to exfiltration in order to make it portable and minimize the amount of data sent over the network

## Rule Content
```
- title: Data Compressed
  id: a3b5e3e9-1b49-4119-8b8e-0344a01f21ee
  status: experimental
  description: An adversary may compress data (e.g., sensitive documents) that is
    collected prior to exfiltration in order to make it portable and minimize the
    amount of data sent over the network
  author: Timur Zinniatullin, oscd.community
  date: 2019/10/21
  modified: 2019/11/04
  references:
  - https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1002/T1002.yaml
  logsource:
    product: linux
    service: auditd
    category: null
  detection:
    selection1:
      type: execve
      a0: zip
    selection2:
      type: execve
      a0: gzip
      a1: -f
    selection3:
      type: execve
      a0: tar
      a1|contains: -c
    condition: 1 of them
  falsepositives:
  - Legitimate use of archiving tools by legitimate user
  level: low
  tags:
  - attack.exfiltration
  - attack.t1002

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(type:"execve" AND (a0:"zip" OR (a0:"gzip" AND a1:"\-f") OR (a0:"tar" AND a1.keyword:*\-c*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()